In [ ]:
# Setup
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from math import sqrt
from sklearn.preprocessing import MinMaxScaler

import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# supress warnings
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load Dataset
df = pd.read_csv('/content/drive/MyDrive/Bangkit/Dataset/AllProductPriceDataset.csv', header=0, index_col=0, parse_dates=True)
df

,Bayam,Kangkung,Sawi,Brokoli,Kubis,Wortel,Kentang,Kacang Polong,Kacang Tanah,Buncis,...,Garu Piring,Lada,Cengkeh,Kayu Manis,Kapulaga,Kunyit,Ubi Jalar,Ubi Kayu,Singkong,Talas
Tanggal,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1500,1000,5500,9000,3500,8000,6000,30000,26000,17000,...,5000,89000,130000,17000,83000,3500,3000,4000,1000,14000
2013-08-01,1500,1000,5500,9000,3500,8000,6000,30000,26000,17000,...,5000,89000,130000,17000,83000,3500,3000,4000,1000,14000
2013-01-15,1500,1000,5500,9000,3500,8000,6000,30000,26000,16000,...,5000,89000,130000,17000,83000,3500,3000,4000,1000,14000
2013-01-22,1500,1000,5500,9000,3500,8000,6000,30000,26000,16000,...,5000,89000,130000,17000,83000,3500,3000,4000,1000,14000
2013-01-29,1500,1000,5500,9000,3500,8000,6000,30000,26000,16000,...,5000,89000,130000,17000,83000,3500,3000,4000,1000,14000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-27,14000,13500,7500,35500,14000,16000,15000,58500,30000,33500,...,9000,149000,199000,22500,87000,9000,9500,6000,2000,24500
2023-03-11,14000,13500,7500,35500,14000,16000,15000,58500,30000,33500,...,9000,149000,199000,22500,87000,9000,9500,6000,2000,24500
2023-10-11,14000,13500,7500,35500,14000,16000,15000,58500,30000,33500,...,9000,149000,199000,22500,87000,9000,9500,6000,2000,24500


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 570 entries, 2013-01-01 to 2023-11-24
Data columns (total 53 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Bayam                 570 non-null    int64
 1   Kangkung              570 non-null    int64
 2   Sawi                  570 non-null    int64
 3   Brokoli               570 non-null    int64
 4   Kubis                 570 non-null    int64
 5   Wortel                570 non-null    int64
 6   Kentang               570 non-null    int64
 7   Kacang Polong         570 non-null    int64
 8   Kacang Tanah          570 non-null    int64
 9   Buncis                570 non-null    int64
 10  Labu Siam             570 non-null    int64
 11  Labu Kuning           570 non-null    int64
 12  Lobak Merah           570 non-null    int64
 13  Lobak Putih           570 non-null    int64
 14  Tomat                 570 non-null    int64
 15  Terong                570 non-null    

In [ ]:
# Fungsi untuk Membentuk Data untuk Model
def df_to_X_y(df, window_size=75):
      df_as_np = df.to_numpy()
      X = []
      y = []

      for i in range(len(df_as_np)-window_size):
          row = [[a] for a in df_as_np[i:i+window_size]]
          X.append(row)
          label = df_as_np[i+window_size]
          y.append(label)
      return np.array(X), np.array(y)

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
    def __init__(self, target_rmse=0.01):
        super(MyCallback, self).__init__()
        self.target_rmse = target_rmse

    def on_epoch_end(self, epoch, logs=None):
        val_rmse = logs.get('val_root_mean_squared_error')
        if val_rmse is not None and val_rmse < self.target_rmse:
            print(f"\nReached target Validation RMSE ({self.target_rmse}), stopping training.")
            self.model.stop_training = True

In [ ]:
jumlah_kolom = df.shape[1]
jumlah_kolom

53

In [ ]:
# min 0, max 53
awal = 10
akhir = 30
for i in range(awal,akhir):
  nama_produk = df.columns[i]
  nama_produk = nama_produk.replace(" ", "_")
  print(nama_produk)

Labu_Siam
Labu_Kuning
Lobak_Merah
Lobak_Putih
Tomat
Terong
Paprika_Hijau
Paprika_Kuning
Paprika_Merah
Bawang_Bombay
Bawang_Putih
Bawang_Merah
Cabai_Keriting_Merah
Cabai_Keriting_Hijau
Cabai_Rawit_Hijau
Cabai_Rawit_Merah
Cabai_Rawit_Kering
Mentimun
Pare_Paria
Jamur


In [ ]:
# Inisialisasi dictionary untuk menyimpan hasil evaluasi
evaluasi_produk = {}

# Fungsi untuk menambahkan hasil evaluasi ke dictionary
def tambahkan_evaluasi(nama_produk, y_train, train_predict, y_test, test_predict):
    evaluasi_produk[nama_produk] = {
        'Train RMSE': math.sqrt(mean_squared_error(y_train, train_predict)),
        'Train MSE': mean_squared_error(y_train, train_predict),
        'Train MAE': mean_absolute_error(y_train, train_predict),
        'Test RMSE': math.sqrt(mean_squared_error(y_test, test_predict)),
        'Test MSE': mean_squared_error(y_test, test_predict),
        'Test MAE': mean_absolute_error(y_test, test_predict),
    }

In [ ]:
# jumlah_kolom = df.shape[1]
# jumlah_kolom
# train model
for i in range(awal,akhir):
  nama_produk = df.columns[i]
  series_harga = df[nama_produk]

  # Normalisasi Data
  mean_value = series_harga.mean()
  std_value = series_harga.std()
  normalized_series = (series_harga - mean_value) / std_value

  WINDOW_SIZE = 5
  X, y = df_to_X_y(normalized_series, WINDOW_SIZE)
  X.shape, y.shape

  # Pembagian Data menjadi Train, Validation, dan Test sets
  X_train, y_train = X[:510], y[:510]
  X_val, y_val = X[510:530], y[510:530]
  X_test, y_test = X[530:], y[530:]

  # Inisialisasi scaler
  scaler = MinMaxScaler(feature_range=(0, 1))

  # Normalisasi X_train
  X_train = scaler.fit_transform(X_train.reshape(-1, WINDOW_SIZE))

  # Normalisasi X_val
  X_val = scaler.transform(X_val.reshape(-1, WINDOW_SIZE))

  # Normalisasi X_test
  X_test = scaler.transform(X_test.reshape(-1, WINDOW_SIZE))

  model = tf.keras.Sequential([
      tf.keras.layers.InputLayer(input_shape=(WINDOW_SIZE, 1)),
      tf.keras.layers.LSTM(64, return_sequences='True'),
      tf.keras.layers.LSTM(32),
      tf.keras.layers.Dense(8, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])

  # Kompilasi dan Pelatihan Model dengan Normalisasi
  # checkpoint = tf.keras.callbacks.ModelCheckpoint('model/', save_best_only=True)
  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  callbacks = MyCallback(target_rmse=0.01)
  model.fit(X_train,
            y_train,
            validation_data=(X_val, y_val),
            epochs=1000,
            callbacks=[callbacks])

  train_predict=model.predict(X_train)
  test_predict=model.predict(X_test)

  print("\n\n\nTrain data RMSE ", nama_produk," : ", math.sqrt(mean_squared_error(y_train,train_predict)))
  print("Train data MSE " , nama_produk," : ", mean_squared_error(y_train,train_predict))
  print("Train data MAE " , nama_produk," : ", mean_absolute_error(y_train,train_predict))
  print("-------------------------------------------------------------------------------------")
  print("Test data RMSE " , nama_produk," : ", math.sqrt(mean_squared_error(y_test,test_predict)))
  print("Test data MSE "  , nama_produk," : ", mean_squared_error(y_test,test_predict))
  print("Test data MAE "  , nama_produk," : ", mean_absolute_error(y_test,test_predict))
  print("\n\n\n")

  nama_produk = nama_produk.replace(" ", "_")
  tambahkan_evaluasi(nama_produk, y_train, train_predict, y_test, test_predict)

  # Save your model in the SavedModel format
  model.save(f"model_{nama_produk}.h5")

Output streaming akan dipotong hingga 5000 baris terakhir.
Epoch 525/1000
16/16 [==============================] - 0s 18ms/step - loss: 0.1921 - root_mean_squared_error: 0.4383 - val_loss: 0.0884 - val_root_mean_squared_error: 0.2973
Epoch 526/1000
16/16 [==============================] - 0s 19ms/step - loss: 0.1906 - root_mean_squared_error: 0.4365 - val_loss: 0.0685 - val_root_mean_squared_error: 0.2618
Epoch 527/1000
16/16 [==============================] - 0s 18ms/step - loss: 0.1905 - root_mean_squared_error: 0.4364 - val_loss: 0.1018 - val_root_mean_squared_error: 0.3190
Epoch 528/1000
16/16 [==============================] - 0s 19ms/step - loss: 0.1962 - root_mean_squared_error: 0.4429 - val_loss: 0.0699 - val_root_mean_squared_error: 0.2643
Epoch 529/1000
16/16 [==============================] - 0s 20ms/step - loss: 0.1914 - root_mean_squared_error: 0.4375 - val_loss: 0.0861 - val_root_mean_squared_error: 0.2935
Epoch 530/1000
16/16 [==============================] - 0s 18ms/st

In [ ]:
# Menampilkan hasil evaluasi
for produk, metrik in evaluasi_produk.items():
    print(f"\n\n\n{produk} Metrics:")
    print("Train RMSE:", metrik['Train RMSE'])
    print("Train MSE:", metrik['Train MSE'])
    print("Train MAE:", metrik['Train MAE'])
    print("-------------------------------------------------------------------------------------")
    print("Test RMSE:", metrik['Test RMSE'])
    print("Test MSE:", metrik['Test MSE'])
    print("Test MAE:", metrik['Test MAE'])




Bayam Metrics:
Train RMSE: 0.13310946290877435
Train MSE: 0.017718129115862374
Train MAE: 0.04520490564496106
-------------------------------------------------------------------------------------
Test RMSE: 0.08453324749343658
Test MSE: 0.007145869931786602
Test MAE: 0.02951045493901141



Kangkung Metrics:
Train RMSE: 0.14009983980235954
Train MSE: 0.019627965112646806
Train MAE: 0.058245577996006874
-------------------------------------------------------------------------------------
Test RMSE: 0.12763497987944722
Test MSE: 0.0162906880888269
Test MAE: 0.09045183981070694



Sawi Metrics:
Train RMSE: 0.2673474953376319
Train MSE: 0.07147468326330511
Train MAE: 0.1044686296392158
-------------------------------------------------------------------------------------
Test RMSE: 0.09115123610226394
Test MSE: 0.008308547842970665
Test MAE: 0.04736271124043533



Brokoli Metrics:
Train RMSE: 0.0782707037351428
Train MSE: 0.006126303063194499
Train MAE: 0.03282529452382139
---------------